In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.preprocessing import OneHotEncoder , PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score , mean_squared_error
from sklearn.model_selection import cross_val_score , train_test_split
warnings.filterwarnings('ignore')
train = pd.read_csv('/Users/hajiaga/Downloads/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('/Users/hajiaga/Downloads/house-prices-advanced-regression-techniques/test.csv')


In [19]:
num_col = train._get_numeric_data().columns
string_cols = train.select_dtypes(include='O').columns.tolist()
if 'Id' in train.columns:
    train.drop(["Id"] , inplace=True , axis= 1, errors='ignore') # add errors='ignore' to avoid KeyError
for col in num_col:
    if train[col].isnull().sum()/1460 == 1:
        train.drop(col , inplace=True)
    elif train[col].isnull().sum()/1460 != 0 :
        train[col].fillna(0, inplace=True)
for col in string_cols:
    if train[col].isnull().sum()/1460 == 1:
        train.drop(col , inplace=True)
    elif train[col].isnull().sum()/1460 != 0 :
        train[col].fillna("0", inplace=True)
train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1460 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   Alley          1460 non-null   object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

In [9]:
def get_outliers(train , col):
    Q1 = train[col].quantile(0.05)
    Q3 = train[col].quantile(0.95)
    IQR = Q3 - Q1
    lower_bound = Q1 - IQR*1.5
    higher_bound = Q3 + IQR*1.5
    return train.loc[(train[col]<lower_bound) | (train[col]>higher_bound)]

In [ ]:
for col in num_col:
    if get_outliers(train , col).shape[0]>0:
        print(col, "#############", get_outliers(train , col))
    sns.scatterplot(train , x = train["SalePrice"] , y = col)
    plt.show()

In [27]:
from sklearn.compose import ColumnTransformer

X = train.drop("SalePrice" , axis=1)
Y = train["SalePrice"]

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), string_cols)], remainder='passthrough')
X = ct.fit_transform(X)


In [12]:
X_train , X_test , Y_train ,Y_test = train_test_split(X , Y , train_size=0.30 , random_state=42)

lr = LinearRegression()
lr.fit( X_train ,  Y_train)
print(r2_score(Y_test , lr.predict(X_test)))
print(r2_score(Y_train , lr.predict(X_train)))

0.790889576838302
0.965645528969308


In [13]:
cvs = cross_val_score(lr , X_train , Y_train , cv = 5 , scoring= "r2")
cvs.mean()


0.8645431892710029

In [28]:
if 'Id' in test.columns:
    test.drop(["Id"] , inplace=True , axis= 1, errors='ignore') # add errors='ignore' to avoid KeyError
num_colo = test._get_numeric_data().columns
string_colso = test.select_dtypes(include='O').columns.tolist()

for col in num_colo:
    if test[col].isnull().sum()/1459 == 1:
        test.drop(col , inplace=True)
    elif test[col].isnull().sum()/1459 != 0 :
        test[col].fillna(0, inplace=True)
for col in string_colso:
    if test[col].isnull().sum()/1459 == 1:
        test.drop(col , inplace=True)
    elif test[col].isnull().sum()/1459 != 0 :
        test[col].fillna("", inplace=True)
# use the same ColumnTransformer object to transform the test set
test = ct.transform(test)

# predict the SalePrice for the test set
print(lr.predict(test))
test.head()



ValueError: Found unknown categories ['0'] in column 0 during transform

In [21]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1459 non-null   int64  
 1   MSSubClass     1459 non-null   int64  
 2   MSZoning       1459 non-null   object 
 3   LotFrontage    1459 non-null   float64
 4   LotArea        1459 non-null   int64  
 5   Street         1459 non-null   object 
 6   Alley          1459 non-null   object 
 7   LotShape       1459 non-null   object 
 8   LandContour    1459 non-null   object 
 9   Utilities      1459 non-null   object 
 10  LotConfig      1459 non-null   object 
 11  LandSlope      1459 non-null   object 
 12  Neighborhood   1459 non-null   object 
 13  Condition1     1459 non-null   object 
 14  Condition2     1459 non-null   object 
 15  BldgType       1459 non-null   object 
 16  HouseStyle     1459 non-null   object 
 17  OverallQual    1459 non-null   int64  
 18  OverallC